In [ ]:
from numpy.ma.core import shape
import random

import numpy as np
import tensorflow as tf

from keras.datasets import mnist
from keras.datasets import fashion_mnist

from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import Dense 

import matplotlib.pyplot as plt 
from keras import layers
from keras import Sequential
from keras.models import  model_from_json
import cv2          

In [ ]:




SEED_VALUE=42
random.seed(SEED_VALUE)
np.random.seed(SEED_VALUE)
tf.random.set_seed(SEED_VALUE)

(x_trainall, y_trainall), (x_test, y_test) = mnist.load_data()
print(x_trainall)
x_valid=x_trainall[:10000]
x_train=x_trainall[10000:]

y_Valid=y_trainall[:10000]
y_train=y_trainall[10000:]

# print(x_train,shape)
# print(x_valid,shape)
# print(x_test,shape)

plt.figure(figsize=(15,5))
for i in range(3):
  plt.subplot(1,3,i+1)
  # plt.axis()
  plt.imshow(x_train[i],cmap='gray')
  plt.subplots_adjust(0.2,0.2)

plt.show()


x_train=x_train.reshape(x_train.shape[0],28*28)
x_train=x_train.astype('float32')/255

x_valid=x_valid.reshape(x_test.shape[0],28*28)
x_valid=x_valid.astype('float32')/255

x_test=x_test.reshape(x_test.shape[0],28*28)
x_test=x_test.astype('float32')/255

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)
y_Valid=to_categorical(y_Valid)


model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(x_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

trainig_result= model.fit(x_train,y_train, batch_size=32, epochs=5, validation_data=(x_valid, y_Valid))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
def getpic(filename):
        img = cv2.imread(filename)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, threshold = cv2.threshold(gray, 110, 255, cv2.THRESH_BINARY)

        contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        rectangles = [cv2.boundingRect(c) for c in contours]
        sorted_rectangles = sorted(rectangles, key=lambda r: r[2]*r[3], reverse=True)
        # max_rect = max(rectangles, key=lambda r: r[2]*r[3])
        sorted_rectangles = sorted(rectangles, key=lambda r: r[2]*r[3], reverse=True)
        max_rects = sorted_rectangles[:2]
        for i, rect in enumerate(max_rects):
          x, y, w, h = rect
          center_x = x + w/2
          center_y = y + h/2
          cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
          cv2.circle(img, (int(center_x), int(center_y)), 5, (0, 0, 255), -1)
          square_img = cv2.resize(img, (440, 440))
          
          cv2.imwrite(f'Imagederect{i}.png', square_img)
        

In [ ]:
def crop(filename):
     # Load the image
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)

    # Define the region of interest (ROI)
    x = 0  # x-coordinate of the top-left corner of the ROI
    y = 0  # y-coordinate of the top-left corner of the ROI
    width = 0 # width of the ROI
    height = 0  # height of the ROI

    # Crop the image
    roi = img[y:y+height, x:x+width]

    # Threshold the image
    thresh_val = 127 
    maxval = 255  
    type = cv2.THRESH_BINARY 
    ret, binary_img = cv2.threshold(roi, thresh_val, maxval, type)
    cv2.imshow( binary_img)


In [ ]:
def testmodel(filename):
    img = cv2.imread(filename, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = img.astype('float32') / 255.0
    img = img.reshape(1, 28*28)
    pred = model.predict(img)
    print(np.argmax(pred))
    class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    pred_class_name = class_names[np.argmax(pred)]
    print('Predicted class:', pred_class_name)

In [ ]:


# تعیین مشخصات وب کم
camera_port = 0
ramp_frames = 30
camera = cv2.VideoCapture(camera_port)

# چک کردن اینکه وب کم را می‌توان دسترسی پیدا کرد یا نه
def get_image():
    # خواندن تعدادی فریم برای آماده سازی کیفیت وب کم
    for i in range(ramp_frames):
        retval, im = camera.read()
    # خواندن یک فریم واقعی از وب کم
    retval, im = camera.read()
    return im

while True:
    # خواندن یک فریم از وب کم
    ret, frame = camera.read()
    
    # نمایش فریم در پنجره
    cv2.imshow('Webcam Feed', frame)
    
    # اگر کاربر کلید q را فشار داد
    if cv2.waitKey(1) & 0xFF == ord('q'):
        # خروج از حلقه بی نهایت و آزاد کردن دوربین
        camera.release()
        cv2.destroyAllWindows()
        break
    
    # اگر کاربر کلید s را فشار داد
    elif cv2.waitKey(1) & 0xFF == ord('s'):
        getpic(frame=frame)
        camera.release()
        cv2.destroyAllWindows()
        break